In [3]:
from bs4 import BeautifulSoup 
import requests

In [690]:
from bs4 import BeautifulSoup 
import requests

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import pandas as pd

from datetime import date, timedelta, datetime
from datetime import timedelta
from datetime import datetime

import smtplib, ssl
from email.message import EmailMessage

In [119]:
options = Options()
options.add_argument('--headless') 
options.add_experimental_option('prefs', {'download.default_directory': os.getcwd()})
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\gama5\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [123]:
from IPython.display import clear_output
from tqdm import tqdm

In [124]:
driver.get("https://www.adorocinema.com/filmes-todos/")
pages = driver.find_elements(By.XPATH, "//a[@class='xXx button button-md item']")
infos = []
# for x in tqdm(range(5000)):
for x in tqdm(range(1, int(pages[-1].text)+1)):
    clear_output(wait=True)
    driver.get(f"https://www.adorocinema.com/filmes-todos/?page={x}")

    sp = BeautifulSoup(driver.page_source, 'html.parser')
    movie_info = sp.findAll('div', class_='meta-body-item meta-body-info')
    movie_title = sp.findAll('a', class_='meta-title-link')

    for movie in range(len(movie_info)):
        movie_info[movie] = movie_info[movie].text.split('/')
        try:
            movie_info[movie] = list(map(lambda x: x.replace("\n", ""), movie_info[movie]))
            movie_info[movie].insert(0, movie_title[movie].text)
            infos.append([movie_info[movie][0], movie_info[movie][1], movie_info[movie][-1]])
        except:
            print("Erro")

100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


#Tratamento de dados

In [643]:
df = pd.DataFrame(infos, columns=['title', 'release_date', 'genre'])

In [644]:
df = df.drop(df[df.release_date == 'em breve'].index)
df = df.drop(df[df.release_date == 'Data de lançamento desconhecida'].index)
df = df.drop(df[df.release_date == ' Data de lançamento desconhecida'].index)
df = df.drop(df[df.release_date == 'Data lançamento sconhecida '].index)
df = df.dropna()

In [645]:
#Selecionando apenas um genero
df['genre'] = df['genre'].str.split(',').str[0]

In [646]:
#Criando backup de csv
# df_export = df.head(50)
# df_export.to_csv('movies_sample.csv', index=False)
# df.to_csv('movies.csv', index=False)

In [647]:
count = df['release_date'].str.split().str.len()
df['mes'] = df['release_date'].str.rsplit(' ',1).str[-2]
df['ano'] = df['release_date'].str.rsplit(' ',1).str[-1]
df['dia'] = df['mes'].str.rsplit(' ',1).str[-2]
df['mes'] = df['mes'].str.replace('\d+', '')
df['mes'] = df['mes'].str.replace('de ', '')
df['mes'] = df['mes'].str.replace(' de', '')
df['dia'] = df['dia'].str.replace(' de', '')
df['dia'] = df['dia'].str.replace(' janeiro', '')
df['dia'] = df['dia'].str.replace(' fevereiro', '')

df['dia'] = df['dia'].str.replace(' março', '')

df['dia'] = df['dia'].str.replace(' abril', '')
df['dia'] = df['dia'].str.replace(' maio', '')
df['dia'] = df['dia'].str.replace(' junho', '')
df['dia'] = df['dia'].str.replace(' julho', '')
df['dia'] = df['dia'].str.replace(' agosto', '')
df['dia'] = df['dia'].str.replace(' setembro', '')
df['dia'] = df['dia'].str.replace(' outubro', '')
df['dia'] = df['dia'].str.replace(' novembro', '')
df['dia'] = df['dia'].str.replace(' dezembro', '')
df['mes'] = df['mes'].str.replace('zembro', 'dezembro')
df['dia'] = df['dia'].str.replace('zembro', '')


<ipython-input-647-12e6e07c1113>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['mes'] = df['mes'].str.replace('\d+', '')


In [648]:
def new_column1(row):
  if row['mes'] ==' janeiro':
    return '01'
  elif row['mes'] ==' fevereiro': 
    return '02' 
  elif row['mes'] ==' março': 
    return '03' 
  elif row['mes'] ==' abril': 
    return '04'
  elif row['mes'] ==' maio': 
    return '05' 
  elif row['mes'] ==' junho': 
    return '06'
  elif row['mes'] ==' julho': 
    return '07'
  elif row['mes'] =='agosto': 
    return '08'
  elif row['mes'] =='setembro': 
    return '09' 
  elif row['mes'] =='outubro': 
    return '10' 
  elif row['mes'] =='novembro': 
    return '11' 
  elif row['mes'] =='dezembro': 
    return '12' 

In [649]:
df['mes_2'] = df.apply(new_column1, axis=1)
import numpy as np
df = df.replace(to_replace='None', value=np.nan).dropna()
df['date_release'] = df['ano'] + '-' + df['mes_2'] + '-' + df['dia']
df['date_release'] = pd.to_datetime(df['date_release'])
df = df[['title', 'date_release', 'genre']]
df.reset_index(drop=True)

,title,date_release,genre
0,Top Gun: Maverick,2022-05-26,Ação
1,Doutor Estranho no Multiverso da Loucura,2022-05-05,Fantasia
2,Sonic 2 - O Filme,2022-04-07,Animação
3,Top Gun - Ases Indomáveis,1986-07-10,Ação
4,Jurassic World: Domínio,2022-06-02,Ação
...,...,...,...
8658,O Novo Testamento,2020-06-12,Comédia
8659,O Ornitólogo,2017-03-30,Drama
8660,O Outro Lado do Crime,1979-04-02,Crime
8661,O Pacto de Adriana,2018-01-11,Documentário


In [650]:
#Convertendo a coluna date_release para data
df['date_release'] = pd.to_datetime(df['date_release'], format='%Y/%m/%d')
df['date_release'] = pd.to_datetime(df['date_release'])

In [651]:
lista_tipo = []
for i in range(df.shape[0]):
  lista_tipo.append(0)
df['n_orders'] = lista_tipo 

In [652]:
#Montando datas de 6 meses antes e depois
date_before_6_months = date.today()+ timedelta(days=-(30*6))
date_after_6_months = date.today()+ timedelta(days=30*6)
today = date.today()

date_before_6_months = date_before_6_months.strftime('%Y-%m-%d')
date_after_6_months = date_after_6_months.strftime('%Y-%m-%d')
today = today.strftime('%Y-%m-%d')

In [653]:
#Criando DF de 6 meses antes de depois

df_SixMonthsBefore = df[(df['date_release'] > date_before_6_months) & (df['date_release'] < today)]
df_SixMonthsAfter = df[(df['date_release'] > today) & (df['date_release'] < date_after_6_months)]

In [654]:
#Ordenando DF por ordem de lançamento
df_SixMonthsBefore.sort_values(by=['date_release'], ascending=False)
df_SixMonthsAfter.sort_values(by=['date_release'], ascending=False)

,title,date_release,genre,n_orders
4435,Vengeance,2022-07-29,Suspense,0
865,Trem-Bala,2022-07-29,Ação,0
75,DC Liga Dos Super Pets,2022-07-28,Animação,0
507,Agente Oculto,2022-07-22,Suspense,0
1517,Um Lugar Bem Longe Daqui,2022-07-15,Drama,0
...,...,...,...,...
517,A Boa Mãe,2022-06-02,Drama,0
419,Está Tudo Bem,2022-06-02,Comédia dramática,0
574,45 do Segundo Tempo,2022-06-02,Comédia,0
868,1982,2022-06-02,Drama,0


In [655]:
#Criando colunas de ID
lista_tipo = []
for i in range(df_SixMonthsBefore.shape[0]):
  lista_tipo.append(i)
df_SixMonthsBefore['id'] = lista_tipo 

lista_tipo = []
for i in range(df_SixMonthsAfter.shape[0]):
  lista_tipo.append(i)
df_SixMonthsAfter['id'] = lista_tipo 

<ipython-input-655-24150b1ce45f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SixMonthsBefore['id'] = lista_tipo
<ipython-input-655-24150b1ce45f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SixMonthsAfter['id'] = lista_tipo


In [656]:
#Criando coluna n_orders com valor 0
lista_tipo = []
for i in range(df.shape[0]):
  lista_tipo.append(0)
df['n_orders'] = lista_tipo 

In [657]:
SixMonthsAfter = {'id': df_SixMonthsAfter['id'],
         'title': df_SixMonthsAfter['title'],
         'genre': df_SixMonthsAfter['genre'],
         'release_date': df_SixMonthsAfter['date_release'],
         'n_orders': df_SixMonthsAfter['n_orders']
         }

dfafter = pd.DataFrame(SixMonthsAfter, columns= ['id','title', 'genre', 'release_date', 'n_orders'])

In [660]:
SixMonthsBefore = {'id': df_SixMonthsBefore['id'],
         'title': df_SixMonthsBefore['title'],
         'genre': df_SixMonthsBefore['genre'],
         'release_date': df_SixMonthsBefore['date_release'],
         'n_orders': df_SixMonthsBefore['n_orders']
         }

dfbefore = pd.DataFrame(SixMonthsBefore, columns= ['id','title', 'genre', 'release_date', 'n_orders'])

#Criando Banco de Dados

In [661]:
import pandas as pd
import sqlite3

In [665]:
#Criando base de dados e cursor
conn = sqlite3.connect('movies.db', check_same_thread=False)
c = conn.cursor()

In [666]:
#Criando tabelas
c.execute('CREATE TABLE IF NOT EXISTS SixMonthsAfter (id, title, genre, release_date, n_orders)')
c.execute('CREATE TABLE IF NOT EXISTS SixMonthsBefore (id, title, genre, release_date, n_orders)')
conn.commit()

In [667]:
#Convertendo dataframe em tabelas
dfafter.to_sql('SixMonthsAfter', conn, if_exists='replace', index = False)
dfbefore.to_sql('SixMonthsBefore', conn, if_exists='replace', index = False)

In [668]:
#Imprimindo consulta(tudo)
for row in c.execute('''SELECT * FROM SixMonthsAfter'''):
  print (row)

(0, 'Jurassic World: Domínio', 'Ação', '2022-06-02 00:00:00', 0)
(1, 'Thor: Amor e Trovão', 'Aventura', '2022-07-07 00:00:00', 0)
(2, 'Elvis', 'Biografia', '2022-07-14 00:00:00', 0)
(3, 'Tudo em Todo Lugar ao Mesmo Tempo', 'Ficção científica', '2022-06-23 00:00:00', 0)
(4, 'Assassino Sem Rastro', 'Suspense', '2022-06-09 00:00:00', 0)
(5, 'Lightyear', 'Animação', '2022-06-16 00:00:00', 0)
(6, 'DC Liga Dos Super Pets', 'Animação', '2022-07-28 00:00:00', 0)
(7, 'Amor & Gelato', 'Comédia', '2022-06-22 00:00:00', 0)
(8, 'Má Sorte no Sexo ou no Pornô Acidental', 'Comédia', '2022-06-02 00:00:00', 0)
(9, 'O Telefone Preto', 'Terror', '2022-06-23 00:00:00', 0)
(10, 'Amado', 'Drama', '2022-06-09 00:00:00', 0)
(11, 'Está Tudo Bem', 'Comédia dramática', '2022-06-02 00:00:00', 0)
(12, 'Agente Oculto', 'Suspense', '2022-07-22 00:00:00', 0)
(13, 'A Boa Mãe', 'Drama', '2022-06-02 00:00:00', 0)
(14, '45 do Segundo Tempo', 'Comédia', '2022-06-02 00:00:00', 0)
(15, 'Um Broto Legal', 'Biografia', '2022-06

In [669]:
#Imprimindo consulta(tudo)
for row in c.execute('''SELECT * FROM SixMonthsBefore'''):
  print (row)

(0, 'Top Gun: Maverick', 'Ação', '2022-05-26 00:00:00', 0)
(1, 'Doutor Estranho no Multiverso da Loucura', 'Fantasia', '2022-05-05 00:00:00', 0)
(2, 'Sonic 2 - O Filme', 'Animação', '2022-04-07 00:00:00', 0)
(3, 'Combinação Perfeita', 'Comédia', '2022-05-19 00:00:00', 0)
(4, 'O Homem do Norte', 'Ação', '2022-05-12 00:00:00', 0)
(5, 'Animais Fantásticos: Os Segredos de Dumbledore', 'Fantasia', '2022-04-14 00:00:00', 0)
(6, '365 Dias: Hoje', 'Drama', '2022-04-27 00:00:00', 0)
(7, 'Luta pela Fé - A História do Padre Stu', 'Drama', '2022-05-26 00:00:00', 0)
(8, 'A Médium', 'Terror', '2022-05-19 00:00:00', 0)
(9, 'Homem-Aranha: Sem Volta para Casa', 'Ação', '2021-12-16 00:00:00', 0)
(10, 'Dog - A Aventura de Uma Vida', 'Comédia dramática', '2022-05-19 00:00:00', 0)
(11, 'Morbius', 'Ação', '2022-03-31 00:00:00', 0)
(12, 'Respire Fundo', 'Drama', '2022-05-06 00:00:00', 0)
(13, 'Chamas da Vingança', 'Suspense', '2022-05-19 00:00:00', 0)
(14, 'Instinto Assassino', 'Ação', '2022-05-18 00:00:00',

#Parte do email

In [734]:
#Capturando lista de filmes que serão lançado daqui 30 dias
thirty_days_to = pd.DataFrame(None)
i=-1
for time in df_SixMonthsAfter['date_release']:
  send = True if (time == (date.today()+ timedelta(days=8))) else False #Adicionando uma margem de erro no dia do lançamento por questões de fuso horario
  i+=1
  if send:
    thirty_days_to = thirty_days_to.append(df_SixMonthsAfter[(df_SixMonthsAfter['id'] == i)])

#Capturando nome dos filmes
title = ""
if send:
  if len((thirty_days_to['title'].tolist()))>1:
    i = len((thirty_days_to['title'].tolist()))
    for movie in (thirty_days_to['title'].tolist()):    
        title += f" {movie},"
  else:
    title = str(thirty_days_to['title'].tolist())

  #Criando assunto do email
  message = f"Mensagem"
  subject = f"Faltam 30 dias para:{title}!"

In [743]:
if send:
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"  # Gmail server
    sender_email = "andre2tiar@gmail.com"  # Enter your address
    receiver_email = "gama595@gmail.com"  # Enter receiver address
    password = 'AndreFiapIA' # Enter your password

    msg = EmailMessage()
    msg.set_content(message)

    msg['From'] = sender_email
    msg['To'] = receiver_email

    #Email subject
    msg['Subject'] = subject

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.send_message(msg)

#Sistema de Recomendação

In [677]:
def getRecommendedMovies(movie):
  recommended_movies = []
  recommendation_list = ""

  #Pesquisando o filme no banco e pegando seu genero
  movie_genre = c.execute('''SELECT genre FROM SixMon WHERE title = ?''', (movie,)).fetchone() 
  movie_genre = ''.join(movie_genre)

  #Pesquisando 5 filmes com o mesmo genero
  for row in c.execute('''SELECT title FROM SixMonthsBefore WHERE genre = ? ORDER BY n_orders ASC LIMIT 5''', (str(movie_genre),)):
    row = ''.join(row)
    recommended_movies.append(row)

  #Convertendo lista de filmes recomendados para HTML
  for movies in recommended_movies:
    recommendation_list += ('''<p style="color: #fff; margin: 0;"> {}</p>'''.format(movies))
    
  return recommendation_list

#Criando API e Funções

In [673]:
def add_order(movie):
  #Pesquisando o filme no banco
  c.execute('''SELECT * FROM SixMonthsAfter where title = ?''', (movie,))
  #Se houver resultado, adiciona 1 ao numero de ordens e retorna sucesso, senão retorna erro
  if c.fetchone():
    c.execute('''UPDATE SixMonthsAfter SET n_orders = n_orders + 1 WHERE title = ?''', (movie,))
    getRecommendedMovies(movie)
    conn.commit()
    return True
  return False

In [678]:
from flask import Flask, request

app = Flask(__name__)

@app.route("/make-order", methods=["GET", "POST"])
def make_order():
    #Metodo POST
    if request.method == "POST":
        movie = request.form.get("movie")

        #Sucesso ao encontra filme
        if add_order(movie):
            return '''
                <div style="position: absolute;
                left: 50vw;
                top: 50vh;
                background: #353535;
                transform: translate(-50%, -50%);
                border: 5px solid #888888;
                padding: 100px;
                display: flex;
                align-items: center;
                flex-direction: column;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;">
                    <h1 style="color: #fff;">Pedido realizado</br> {}</h1>
                    <input type="button" value="Voltar" onclick="history.back()" style="width: 30%; margin: 0.5rem 0.5rem 1rem 0.5rem; padding: 0.5rem;">   
                    <h3 style="color: #fff; margin: 0.5rem 0;"> Voce também pode gostar de:</h3>
                    {}
                </div>
            '''.format(movie, getRecommendedMovies(movie))
        
        #Erro ao encontra filme
        else:
            return  '''                
                <div style="position: absolute;
                left: 50vw;
                top: 50vh;
                background: #353535;
                transform: translate(-50%, -50%);
                border: 5px solid #888888;
                padding: 100px;
                display: flex;
                align-items: center;
                flex-direction: column;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;">
                    <h1 style="color: #fff;">Erro ao fazer o pedido</h1>
                    <input type="button" value="Voltar!" onclick="history.back()" style="width: 30%; padding: 0.5rem;">   
                </div>
            '''

    #Metodo GET
    return '''
    <div style="position: absolute;
    left: 50vw;
    top: 50vh;
    background: #353535;
    transform: translate(-50%, -50%);
    border: 5px solid #888888;
    padding: 100px;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;">
        <h1 style="color: #fff;">Reserve o seu filme agora!</h1>
            <form method="POST" style="display: flex; align-items: center; flex-direction: column;">
                <input type="text" name="movie" placeholder="Nome do filme" style="width: 80%; justify-content:center; margin-bottom: 1rem; padding: 0.5rem;">
                <input type="submit" value="Reservar" style="width: 30%; padding: 0.5rem;">
            </from>
    </div>
    '''

#Iniciando API
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5051)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5051/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jun/2022 22:29:29] "POST /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:36] "GET /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:39] "POST /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:43] "GET /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:45] "POST /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:46] "GET /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:55] "POST /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:29:59] "GET /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:30:00] "POST /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:30:01] "GET /make-order HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 22:30:07] "POST /make-order HTTP/1.1" 200 -
